In [14]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from tqdm.notebook import tqdm

In [19]:
train_df = pd.read_csv('data_full/train.csv')
train_ed = pd.read_csv('data_full/trainEducationFeatures.csv')
train_groups_df = pd.read_csv('data_full/trainGroups.csv')
friends_df = pd.read_csv('data_full/friends.csv')
train_df = train_df.merge(train_ed)

In [20]:
everyone = set(np.concatenate((friends_df.uid.unique(), friends_df.fuid.unique(), train_groups_df.uid.unique(), train_df.uid.unique())))
groups_dic = {str(x):[] for x in train_groups_df.gid.values}
users_dic = {str(x): {
    'age': np.nan,
    'registered_year': np.nan,
    'school_education': np.nan,
    'graduation_1': np.nan,
    'graduation_2': np.nan,
    'graduation_3': np.nan,
    'graduation_4': np.nan,
    'graduation_5': np.nan,
    'graduation_6': np.nan,
    'graduation_7': np.nan,
    'friends': [],
    'groups': []
} for x in everyone}

In [21]:
columns = ['registered_year', 'school_education', 'graduation_1', "graduation_2", 'graduation_3', 'graduation_4', 'graduation_5', 'graduation_6', 'graduation_7']
start_values = [2005, 2039, 2043, 2044, 2045, 2046, 2047, 2048, 2049]
uids, ages, registred_years = train_df.uid.values, train_df.age.values, train_df.registered_year.values
for i in range(len(train_df)):
    users_dic[str(uids[i])]['age'] = ages[i]
    for j in range(len(columns)):
        users_dic[str(uids[i])][columns[j]] = abs(start_values[j] - train_df[columns[j]].values[i])
        
        
uids, fuids = friends_df.uid.values.tolist(), friends_df.fuid.values.tolist()
for i in tqdm(range(len(friends_df))):
    users_dic[str(uids[i])]['friends'].append(fuids[i])
    users_dic[str(fuids[i])]['friends'].append(uids[i])
del uids, fuids
    
uids, gids = train_groups_df.uid.values.tolist(), train_groups_df.gid.values.tolist()
for i in tqdm(range(len(train_groups_df))):
    groups_dic[str(gids[i])].append(str(uids[i]))
    users_dic[str(uids[i])]['groups'].append(str(gids[i]))

In [22]:
import pickle
with open('dicts/users_dict_all.pkl', 'wb') as f:
    pickle.dump(users_dic, f)
with open('dicts/group_dict_all.pkl', 'wb') as f:
    pickle.dump(groups_dic, f)